In [1]:
from openfisca_uk import Microsimulation
from openfisca_uk import *
from openfisca_core.model_api import Reform
from openfisca_uk.entities import Person, BenUnit, Household
from openfisca_core.model_api import *
from openfisca_uk.tools.general import *

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

libyaml is not installed in your environment. This can make OpenFisca slower to start. Once you have installed libyaml, run 'pip uninstall pyyaml && pip install pyyaml --no-cache-dir' so that it is used in your Python environment.



In [21]:
sim = Microsimulation(input_year=2020)

New way of creating the reform

In [22]:
from openfisca_core import periods
def make_PA_reform(PA_amount):
    
    def update_PA_parameter(parameters):
        parameters.tax.income_tax.allowances.personal_allowance.amount.update(period=periods.period("year:2020:1"), value=PA_amount)
        return parameters
    
    class reform(Reform):
        def apply(self):
            self.modify_parameters(update_PA_parameter)
    
    sim_less_PA = Microsimulation(reform, input_year=2020)
    revenue = sim.calc("net_income").sum()
    revenue_diff = revenue - sim_less_PA.calc("net_income").sum()
    BI_amount = revenue_diff/(sim.calc("people").sum())
    
    class BI(Variable):
        value_type = float
        entity = Person
        label = u"UBI"
        definition_period = YEAR
        def formula(person, period, parameters):
            return(BI_amount)

    class gross_income(Variable):
        value_type = float
        entity = Person
        label = u"Gross income, including benefits"
        definition_period = YEAR

        def formula(person, period, parameters):
            COMPONENTS = [
                "employment_income",
                "pension_income",
                "self_employment_income",
                "property_income",
                "savings_interest_income",
                "dividend_income",
                "miscellaneous_income",
                "benefits",
                "BI"
            ]
            return add(person, period, COMPONENTS)   

    class basic_income(Reform):
        def apply(self):
            self.add_variable(BI)
            self.update_variable(gross_income)
            
    sim_BI = Microsimulation(reform, basic_income, input_year=2020)
    
    return sim_BI

Effect on poverty at different levels of personal allowance reduction

In [44]:
def find_poverty_diff(sim_BI):
    poverty_difference = sim.calc("in_poverty_bhc", map_to="person").sum() - sim_BI.calc("in_poverty_bhc", map_to="person").sum()
    percentage_difference = (poverty_difference/sim.calc("in_poverty_bhc", map_to="person").sum()) * 100
    return percentage_difference

In [45]:
PA_amounts = [12500, 11500, 10500, 9500, 8500, 7500, 6500, 5500, 4500, 3500, 2500, 1500, 500, 0]
pov_diffs = [find_poverty_diff(make_PA_reform(i)) for i in (PA_amounts)]
pov_diffs

[0.0,
 2.797222727198906,
 4.83694550285811,
 7.430855012549907,
 9.870278329759495,
 12.203120424103577,
 14.608783939890369,
 17.01934063608202,
 18.675456928224797,
 20.79582811839401,
 23.370039171637995,
 25.66720400842633,
 27.99604354406665,
 29.387016516841395]

In [46]:
graph = px.line(x=PA_amounts, y=pov_diffs)
graph.update_layout(
    title_text='Effect of UBI on poverty at different levels of personal allowance reduction',
    xaxis_title ="Amount of personal allowance",
    yaxis_title ="Percentage change in poverty",
)

Original way of creating reform (just deletes personal allowance)

In [ ]:
class PA_reform(Reform):
    def apply(self):
        self.neutralize_variable("personal_allowance")
        
sim_no_PA = Microsimulation(PA_reform)

In [ ]:
revenue = sim.calc("net_income").sum()
revenue_diff = revenue - sim_no_PA.calc("net_income").sum()
revenue_diff

In [ ]:
BI_amount = revenue_diff/(sim.calc("people").sum())
BI_amount

In [ ]:
from openfisca_core.model_api import *
from openfisca_uk.tools.general import *

class BI(Variable):
    value_type = float
    entity = Person
    label = u"UBI"
    definition_period = YEAR
    def formula(person, period, parameters):
        return(BI_amount)

class gross_income(Variable):
    value_type = float
    entity = Person
    label = u"Gross income, including benefits"
    definition_period = YEAR

    def formula(person, period, parameters):
        COMPONENTS = [
            "employment_income",
            "pension_income",
            "self_employment_income",
            "property_income",
            "savings_interest_income",
            "dividend_income",
            "miscellaneous_income",
            "benefits",
            "BI"
        ]
        return add(person, period, COMPONENTS)   

class basic_income(Reform):
    def apply(self):
        self.add_variable(BI)
        self.update_variable(gross_income)
        
sim_BI = Microsimulation(PA_reform, basic_income)

Overall effect on income & taxes

In [34]:
sim_BI = make_PA_reform(0)

In [35]:
sim_BI.calc("net_income").mean() - sim.calc("net_income").mean()

0.000437941311247414

In [36]:
#Difference in median income tax
sim_BI.calc("income_tax").median() - sim.calc("income_tax").median()

1953.9362188333616

In [37]:
#Increase in median household net income
sim_BI.calc("household_net_income").median() - sim.calc("household_net_income").median()

564.157975690181

Effect on each income group

In [38]:
income_diff = sim_BI.calc("household_net_income", map_to="household") - sim.calc("household_net_income", map_to="household")

In [39]:
income = sim.calc("household_net_income", map_to="household")

In [40]:
#Median difference in household net income for each income decile
chart = px.bar(income_diff.groupby(income.decile_rank()).median())
chart.update_layout(
    title_text='Median difference in household net income for each income decile',
    xaxis_title ="Income decile",
    yaxis_title ="Change in income",
)

Effect on poverty

In [ ]:
#Median difference in household net income for those in deep poverty vs those not in deep poverty
isDeepPoor = sim.calc("in_deep_poverty_bhc", map_to ="household")
x = income_diff.groupby(isDeepPoor).median()
x.index = x.index.map({False: "Not in deep poverty", True: "In deep poverty"})
chart2 = px.bar(x)
chart2.update_layout(
    title_text='Median difference in household net income for those in deep poverty vs those not in deep poverty',
    yaxis_title="Change in income",
    xaxis_title="Deep poverty status"
)

In [ ]:
#Median difference in household net income for those in poverty vs those not in poverty
isPoor = sim.calc("in_poverty_bhc", map_to = "household")
x2 = income_diff.groupby(isPoor).median()
x2.index = x2.index.map({False: "Not in poverty", True: "In poverty"})
chart3=px.bar(x2)
chart3.update_layout(
    title_text='Median difference in household net income for those in poverty vs those not in poverty',
    yaxis_title="Change in income",
    xaxis_title="Poverty Status"
)

In [ ]:
#Percentage of poverty reduction
poverty_difference = sim_BI.calc("in_poverty_bhc", map_to="person").sum() - sim.calc("in_poverty_bhc", map_to="person").sum()
(poverty_difference/sim.calc("in_poverty_bhc", map_to="person").sum()) * 100

In [ ]:
#Percentage of deep poverty redution 
deep_poverty_difference = sim_BI.calc("in_deep_poverty_bhc", map_to="person").sum() - sim.calc("in_deep_poverty_bhc", map_to="person").sum()
(deep_poverty_difference/sim.calc("in_deep_poverty_bhc", map_to="person").sum()) * 100

Effect on child poverty

In [ ]:
#Percentage of child poverty reduction
poor_kids_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_child")].sum()
poor_kids_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_child")].sum()
((poor_kids_before - poor_kids_after)/poor_kids_before) * 100

In [48]:
person_income_diff = sim_BI.calc("net_income", map_to="person") - sim.calc("net_income", map_to="person")

In [ ]:
#Mean difference in income by age
age = sim.calc("age", map_to ="person")
chart4 = px.bar(person_income_diff.groupby(age).mean())
chart4.update_layout(
    title_text='Mean difference in income by age',
    yaxis_title="Change in income",
    xaxis_title="Age"
)

In [ ]:
is_child = sim.calc("is_child", map_to ="person")
x3 = person_income_diff.groupby(is_child).mean()
x3.index = x3.index.map({False: "Adult", True: "Child"})
chart5 = px.bar(x3)
chart5.update_layout(
    title_text='Mean difference in income by child vs adult',
    yaxis_title="Change in income",
    xaxis_title="Age group"
)

In [ ]:
#Percentage of poverty reduction for adults only
poor_adults_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_adult")].sum()
poor_adults_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_adult")].sum()
((poor_adults_before - poor_adults_after)/poor_adults_before) * 100

In [ ]:
#Mean income difference grouped by family type
family = sim.calc("family_type", map_to = "person")
chart6 = px.bar(person_income_diff.groupby(family).mean())
chart6.update_layout(
    title_text='Mean income difference grouped by family type',
    yaxis_title="Change in income",
    xaxis_title="Family type"
)